In [1]:
import requests
import base64
import pandas as pd
import psycopg2
from psycopg2 import sql

In [2]:
def get_kroger_access_token(client_id, client_secret):
    url = 'https://api-ce.kroger.com/v1/connect/oauth2/token'
    
    credentials = f"{client_id}:{client_secret}"
    encoded_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')
    
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Authorization': 'Basic ' + encoded_credentials
    }

    data = {
        'grant_type': 'client_credentials',
        'scope': 'product.compact'
    }

    response = requests.post(url, headers=headers, data=data)

    if response.status_code == 200:
        return response.json().get('access_token')
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

access_token = get_kroger_access_token(client_id, client_secret)

In [3]:
def get_kroger_products(access_token, item):
    url = f'https://api-ce.kroger.com/v1/products?filter.term={item}&filter.locationId=01400441'
    
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Cache-Control': 'no-cache'
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None
    
    return data['data']

# Test
item = 'beef'

products_data = get_kroger_products(access_token, item)

products_data

[{'productId': '0001111034567',
  'upc': '0001111034567',
  'aisleLocations': [{'bayNumber': '8',
    'description': 'BACK WALL',
    'number': '152',
    'numberOfFacings': '3',
    'side': 'R',
    'shelfNumber': '1',
    'shelfPositionInBay': '6'}],
  'brand': 'Kroger',
  'categories': ['Meat & Seafood', 'Meat & Seafood', 'Natural & Organic'],
  'description': 'Kroger® Beef Shaved Steak',
  'images': [{'perspective': 'front',
    'featured': True,
    'sizes': [{'size': 'xlarge',
      'url': 'https://www.kroger.com/product/images/xlarge/front/0001111034567'},
     {'size': 'large',
      'url': 'https://www.kroger.com/product/images/large/front/0001111034567'},
     {'size': 'medium',
      'url': 'https://www.kroger.com/product/images/medium/front/0001111034567'},
     {'size': 'small',
      'url': 'https://www.kroger.com/product/images/small/front/0001111034567'},
     {'size': 'thumbnail',
      'url': 'https://www.kroger.com/product/images/thumbnail/front/0001111034567'}]},
  

In [ ]:
conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port
)

cur = conn.cursor()

table_name = 'ingredients'
query = sql.SQL('select distinct "Ingredient" from {}').format(sql.Identifier(table_name))

# Finding weird anomolies in the names and then updating them
# query = sql.SQL("select distinct {} from {} where {} like %s").format(
#     sql.Identifier("Ingredient"),
#     sql.Identifier(table_name),
#     sql.Identifier("Ingredient")
# )

# pattern = '%0.5%'

# cur.execute(query, (pattern,))

cur.execute(query)

rows = cur.fetchall()

for row in rows:
    ingredient = row[0]
    # Getting the ingredient categories
    kroger_data = get_kroger_products(access_token, ingredient)

    if kroger_data:
        categories = [item for item in set(kroger_data[0]['categories'])]

        print(f"{ingredient} - {categories}")
    else:
        print(f"{ingredient} - Add a new category")

In [5]:
cur.close()
conn.close()

In [7]:
# cur = conn.cursor()

# table_name = 'ingredients'
# update_query = sql.SQL('update {} set "Ingredient" = %s where "Ingredient" = %s').format(sql.Identifier(table_name))

# cur.execute(update_query, ('Sabra Classic Hummus', 'Sabra0.5 Classic Hummus'))

# conn.commit()

# cur.close()
# conn.close()